# Tensorflow Low Level API introduction

This notebook introduces the scala_tensorflow library low-level API for a simple regression model case.

Summary:

- load data into a Tensor
- get data back from Tensor to Scala Collections for visualization
- Create a model: Variables, and Placeholders, operations
- Training elements: Loss and Optimizer
- Graph and session
- Tensorboard vizualization
- Model Metrics


## Load tensorflow_scala dependency 

For the `scala_tensorflow` source, see:

https://github.com/eaplatanios/tensorflow_scala 

In [ ]:
interp.load.ivy(coursierapi.Dependency.of("org.platanios", "tensorflow_2.12", "0.4.1").withClassifier("linux-cpu-x86_64"))
interp.load.ivy("org.platanios" %% "tensorflow-data" % "0.4.1")

A few imports to access the tensorflow_scala API

In [ ]:
import org.platanios.tensorflow.api._
import org.platanios.tensorflow.api.learn._
import org.platanios.tensorflow.api.learn.layers._
import org.platanios.tensorflow.api.learn.estimators.InMemoryEstimator
import org.platanios.tensorflow.data.image.MNISTLoader
import org.platanios.tensorflow.api.core.client.FeedMap

import java.nio.file.Paths
import scala.util.Random

In [ ]:
import sys.process._

### Load a simple dataset

Machine learning is a data-driven process, let's load some simple data.

This is a csv with 2 columns, `Price` and `Surface` as double values.

We will want to get a model to predict land `Price` from `Surface`, a linear model should make the cut!

In [ ]:
val rootDir = "resources/"  //sys.env("HOME") + "/data/immo/"

In [ ]:
s"head ${rootDir}land-price.csv"!

#### CSV parsing to Tensors, in pure Scala

We read the file line by line

Skip the header line

Split each row and convert to Double

Then we create a Tensor of Rank 1 (1-d array with 2 elements) for each row:

In [ ]:
val bufferedSource = scala.io.Source.fromFile(s"${rootDir}land-price.csv")
lazy val tt = bufferedSource.getLines.drop(1).toVector
                       .map(_.split(",").map(_.trim.toFloat))
                       .map(arr => Tensor(arr(0), arr(1)))

We then add one dimension to the Tensor by collecting all records in a new one:

In [ ]:
val data = Tensor(tt: _*)

Note the shape, 19 elements of 2 values:

In [ ]:
data.shape

Here are a few examples of loading Tensor values back into scala:

In [ ]:
(data(0,0).scalar, data(0,1).scalar)

In [ ]:
data(0).entriesIterator.toSeq

In [ ]:
data.entriesIterator.toSeq

## Visualize data

In [ ]:
import $ivy.`org.plotly-scala::plotly-almond:0.5.2`
import plotly._, plotly.element._, plotly.layout._, plotly.Almond._

In [ ]:
lazy val trace2 = Scatter(
  data(---, 1).entriesIterator.toSeq,
  data(---, 0).entriesIterator.toSeq,
  mode = ScatterMode(ScatterMode.Markers)
)
lazy val dataPlt = Seq(trace2)

lazy val layout = Layout(
  title = "Scatter Plot"
)



In [ ]:
plot(dataPlt, layout)

## Build a linear model:


In [ ]:
val inputs      = tf.placeholder[Float](Shape(-1, 1))
val outputs     = tf.placeholder[Float](Shape(-1, 1))

In [ ]:
val (predictions, weights) = tf.nameScope("Linear") {
  val weights = tf.variable[Float]("weights", Shape(1, 1), tf.ZerosInitializer)
  (tf.matmul(inputs, weights), weights)
}

In [ ]:
val loss        = tf.sum(tf.square(predictions - outputs))
val optimizer   = tf.train.AdaGrad(1.0f)
val trainOp     = optimizer.minimize(loss)

In [ ]:
val sess = Session()

In [ ]:
sess.run(targets = tf.globalVariablesInitializer())

In [ ]:
val inTensor = data(---, 1).expandDims(-1)
val outTensor = data(---, 0).expandDims(-1)

In [ ]:
val feedMap = Seq(
    FeedMap(inputs, inTensor),
    FeedMap(outputs, outTensor)
    )

In [ ]:
val (currentLoss, currentWeights) = sess.run(fetches = (loss, weights.value), 
                                              targets = Set(trainOp), feeds = feedMap)

In [ ]:
currentWeights.entriesIterator.toSeq

In [ ]:
sess.graph.toMetaGraphDef()

In [ ]:
//val path = java.nio.file.Paths.get("/tmp/000tfsc")
//tf.summary.FileWriter(path, sess.graph)


In [ ]:
val preds = sess.run(fetches = predictions, feeds = FeedMap(inputs -> inTensor))

In [ ]:
preds(---, 0).entriesIterator.toSeq.zip(data(---, 0).entriesIterator.toSeq)

## Plot Predictions vs Actual

In [ ]:
lazy val trace2 = Scatter(
  data(---, 1).entriesIterator.toSeq,
  data(---, 0).entriesIterator.toSeq,
  mode = ScatterMode(ScatterMode.Markers)
)

lazy val trace1 = Scatter(
  data(---, 1).entriesIterator.toSeq,
  preds(---, 0).entriesIterator.toSeq,
  mode = ScatterMode(ScatterMode.Lines)
)

lazy val dataPlt = Seq(trace1, trace2)

lazy val layout = Layout(
  title = "Scatter Plot"
)


In [ ]:
plot(dataPlt, layout)